Load necessary libraries

In [ ]:
### install libraries
#!pip install -q trankit==1.1.0
#!pip install -q transformers

!pip install trankit
!pip install transformers

See which packages are installed in Python

In [ ]:
import sys
print(sys.path)
pip list -v

Add packages in projappl to path

In [ ]:
sys.path.append("/projappl/project_XXXXXXX/my-python-env/lib/python3.9/site-packages/lib/python3.9/site-packages")

In [ ]:
# initialize the parser model	
import trankit
from trankit import Pipeline, trankit2conllu

parser = Pipeline(lang='finnish', gpu=True) #you can also try other languages
# Swedish sentences and multilingual speeches with both Finnish and Swedish are processed locally

Process parliament speeches that were separated into separate text files and zipped into input_files.zip

In [ ]:
# parse and print to file
import os
import zipfile
# import trankit
# from trankit import Pipeline

# initialize the pipeline
# pipeline = Pipeline('finnish')
# already initialized above

# path to the zip file
zip_path = 'input_files.zip'

# create the input and output directories if they don't exist
if not os.path.exists('input_files'):
    os.makedirs('input_files')
if not os.path.exists('output_files'):
    os.makedirs('output_files')

# unzip the files to the input directory
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall()

# iterate over all the files in the input directory
for filename in os.listdir('input_files'):
    if filename.endswith('.txt'):
        # construct the input and output file paths
        input_path = os.path.join('input_files', filename)
        output_path = os.path.join('output_files', filename[:-4] + '.conllu')
        
        # if the output file already exists, skip this iteration
        if os.path.exists(output_path):
            continue

        # open the input file and read the contents
        with open(input_path, 'r') as fin:
            text = fin.read()

        # process the text with trankit
        parsed = parser(text)
        conllu = trankit2conllu(parsed)

        # write the output to the output file
        with open(output_path, 'w') as fout:
            fout.write(conllu)


# create a zip file of the output directory
with zipfile.ZipFile('output_files.zip', 'w', zipfile.ZIP_DEFLATED) as zip_ref:
    for root, dirs, files in os.walk('output_files'):
        for file in files:
            zip_ref.write(os.path.join(root, file))

In [ ]:
# clean things up
!os.rmdir('input_files')
!os.rmdir('output_files')